In [2]:
import shioaji as sj

print(sj.__version__)

from dotenv import load_dotenv
import os

load_dotenv()


api_key = os.environ['SHIOAJI_API_KEY']
secret = os.environ['SHIOAJI_SECRET']

api = sj.Shioaji(simulation=True) # 模擬模式

api.login(
    api_key=api_key,     
    secret_key=secret   
)
print("Login Success")

1.1.11
Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
Login Success


In [147]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
from enum import Enum

# Import the backtrader platform
import backtrader as bt

class Direction(Enum):
    MANY = 'Many'
    EMPTY = 'Empty'

# Create a Stratey
class MyStrategy(bt.Strategy):
    '''
    Strategy: 先以當沖為主
    1. 若今天開盤價大於昨天收盤價，則一律做多，相反則反之
    以做多情況為例：
    2. 當價格來到（開盤到現在）最小值以上30點，則買進
    3. 當價格來到（買進後到現在）最高點以下30點，則賣出
    每天進場操作一次
    '''
    params = (
        ('offset', 30),
        ('last_day_close_price', 0),
        ('debug', False)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Ticks: (close)
        self.debug = self.params.debug

        # Keep a reference to the "close" line in the data[0] dataseries
        self.cur_price = self.datas[0].close
        print('Initialize data...')
        
        self.open_price = None
        
        self.min_price_since_open = 1e9
        self.min_price_since_buy = 1e9
        self.max_price_since_open = 0
        self.max_price_since_buy = 0

        self.strategy_direction = None

        # Params
        self.offset = self.params.offset
        self.last_close_price = self.params.last_day_close_price


        self.finish = False
        self.holding = False
        

    def UseManyStrategy(self):
        if not self.holding and self.cur_price[0] >= self.min_price_since_open + self.offset :
            self.buy()
            self.log(f'Buy at price {self.cur_price[0]}. min price: {self.min_price_since_open}')

            self.holding = True
        
        elif self.holding and self.cur_price[0] <= self.max_price_since_buy - self.offset:
            self.close()
            self.log(f'Sell at price {self.cur_price[0]}. max price: {self.max_price_since_buy}')

            self.holding = False
            self.finish = True


    def UseEmptyStrategy(self):
        if self.holding and self.cur_price[0] >= self.min_price_since_buy + self.offset:
            self.close()
            self.log(f'Close after price {self.cur_price[0]}. min price: {self.min_price_since_buy}')

            self.holding = False
            self.finish = True
        
        elif not self.holding and self.cur_price[0] <= self.max_price_since_open - self.offset :
            self.sell()
            self.log(f'Sell after price {self.cur_price[0]}. max price: {self.max_price_since_open}')

            self.holding = True
            
            
    def next(self):
        
        self.min_price_since_open = min(self.min_price_since_open, self.cur_price[0])
        self.max_price_since_open = max(self.max_price_since_open, self.cur_price[0])
        
        if self.holding:
            # Buy or sell action in next tick. So the real buy is happened in next tick.
            self.min_price_since_buy = min(self.min_price_since_buy, self.cur_price[0])
            self.max_price_since_buy = max(self.max_price_since_buy, self.cur_price[0])
        
        if self.open_price == None:
            # Decide make many or empty
            self.open_price = self.cur_price[0]
            self.strategy_direction = Direction.MANY if self.open_price > self.last_close_price else Direction.EMPTY

            print(f'Strategy use direction: {self.strategy_direction}')
            return 
        
        if self.finish:
            return

        if self.debug:
            print(f"Current price {self.cur_price[0]}")
            # print(self.position.size)
        
        if self.strategy_direction == Direction.MANY:
            self.UseManyStrategy()
        else:
            self.UseEmptyStrategy()
            

In [163]:
from datetime import datetime, timedelta

# Date for running regression test
backtrading_target_date = '2023-05-05'


DAY_OPEN_TIME = "08:45:00"
DAY_END_TIME = "13:45:00"

NIGHT_OPEN_TIME = "15:00:00"
NIGHT_END_TIME = "05:00:00"

# 先以日盤為主
ticks = api.ticks(
    contract=api.Contracts.Futures.MXF.MXF202309, 
    query_type=sj.constant.TicksQueryType.RangeTime,
    time_start=DAY_OPEN_TIME,
    time_end=DAY_END_TIME,
    date=backtrading_target_date
)

In [166]:
import pandas as pd
df = pd.DataFrame({**ticks})
df.ts = pd.to_datetime(df.ts)
df.head()

,ask_price,ts,volume,tick_type,ask_volume,close,bid_price,bid_volume
0,15088.0,2023-05-05 08:45:38.539,1,1,4,15076.0,15065.0,4
1,15085.0,2023-05-05 08:45:54.900,1,1,1,15085.0,15075.0,4
2,15098.0,2023-05-05 08:45:59.365,2,1,1,15086.0,15080.0,4
3,15096.0,2023-05-05 08:46:26.420,1,2,1,15085.0,15081.0,4
4,15096.0,2023-05-05 08:46:36.088,2,1,1,15096.0,15080.0,4


In [168]:
# Convert the string to a datetime object
date_obj = datetime.strptime(backtrading_target_date, '%Y-%m-%d')

# Calculate the day before the given date
yesterday_date_obj = date_obj - timedelta(days=1)

# If you want to store the date as a string in the same format
yesterday_date_str = yesterday_date_obj.strftime('%Y-%m-%d')

last_day_ticks = api.ticks(
    contract=api.Contracts.Futures.MXF.MXF202309, 
    query_type=sj.constant.TicksQueryType.LastCount,
    date=yesterday_date_str,
    last_cnt=1
)


last_day_close_price = last_day_ticks.close[0]
print(f"Last day close price: {last_day_close_price}")

Last day close price: 15065.0


In [170]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(MyStrategy, offset=30, last_day_close_price=last_day_close_price, debug=False)

# Add the Data Feed to Cerebro
# Currently, set open and close to the tick's close price.
# High and low is not necessary to calculate profit.
data = bt.feeds.PandasData(dataname=df, datetime='ts', 
                           open='close',
                           close='close'
                        #    high='close',
                        #    low='close',
                           )
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(1000000.0)

# cerebro.broker.setcommission(commission=0.001)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())



Starting Portfolio Value: 1000000.00
15105.0
738645.5728716783
Strategy use direction: Direction.MANY
2023-05-05, Buy at price 15110.0. min price: 15076.0
2023-05-05, Sell at price 15118.0. max price: 15150.0
Final Portfolio Value: 1000004.00
